# Degree Centrality Algorithm

### Degree centrality involves counting the number of direct connections a node has. This algorithm helps us find bart stations that are overloaded with connections and likely to be busy and crowded.

## We first import relevant libraries.

In [102]:
import neo4j
import csv
import math
import numpy as np
import pandas as pd
import psycopg2

## We then set up our connection, driver, and session. We also build neo4j methods to construct our graph.

In [103]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()

In [104]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [105]:
session = driver.session(database="neo4j")


In [106]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    
def my_neo4j_betweenness_centrality():
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', {TRACK: {properties: 'weight'}})"
    session.run(query)

    queryReal = """

    CALL gds.betweenness.stream(
        'ds_graph', 
        {relationshipWeightProperty: 'weight'}
    )
    YIELD nodeId, score
    RETURN
        gds.util.asNode(nodeId).name AS name,
        score as betweenness
    ORDER BY betweenness DESC

    """
    my_neo4j_run_query_pandas(queryReal)

    

## We now wipe out the neo4j database and check the number of nodes and relationship to ensure they are both 0.

In [107]:
my_neo4j_wipe_out_database()

In [108]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


## These next cells build out our neo4j graph.
### The graph will be constructed such that there is only one node for every BART station. The relationships between nodes represent connetions betweens stations in both directions for each line. The relationships are weighted by travel time.

### First we create a single node for every BART station.

In [109]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node(station)

In [110]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 0
-------------------------


### Next we create relationships between each adjoining station for every line that those 2 stations are adjoined in. We make 2 relationships for each of these connections such that the connection is 2-way. The relationships are weighted by the travel time between those two stations.

In [113]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [114]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 216
-------------------------


### Finally, we run our degree centrality algorithm to count the number of connections for each BART station.

In [129]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

query = """

CALL gds.degree.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as degree
ORDER BY degree DESC, name

"""

my_neo4j_run_query_pandas(query)

,name,degree
0,16th Street Mission,8.0
1,24th Street Mission,8.0
2,Balboa Park,8.0
3,Civic Center,8.0
4,Embarcadero,8.0
5,Glen Park,8.0
6,Montgomery Street,8.0
7,Powell Street,8.0
8,West Oakland,8.0
9,Coliseum,7.0
